# # FITS File Verification Notebook
# 
# This notebook demonstrates how to use the concat function + verify FITS files, including:
# - Basic file verification
# - Comparison against reference files
# - Concatenation of multiple files

In [1]:
# Standard library imports
from pathlib import Path

# Astropy imports for FITS file handling
from astropy.io import fits
from astropy.table import Table

from padre_meddea.calibration.calibration import concatenate_daily_fits

/home/damian/.conda/envs/meddea/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ## Verification Function
# 
# The `verify_fits_file` function performs several checks:
# - Basic file information
# - Table row counts
# - Comparison with reference files (if provided)

In [2]:
def verify_fits_file(test_file, reference_files=None):
    """
    Verify the contents of a FITS file, optionally against reference files.
    
    Args:
        test_file: Path to the FITS file to verify
        reference_files: Single file path or list of files to compare against (optional)
    """
    # Initialize counters
    actual_counts = {}
    expected_counts = {}
    
    # Convert test_file to Path object if it's a string
    test_file = Path(test_file)
    
    # Process reference files first if provided
    if reference_files:
        # Convert single file to list for uniform handling
        if isinstance(reference_files, (str, Path)):
            reference_files = [reference_files]
        
        print(f"\nAnalyzing {len(reference_files)} reference files...")
        
        for ref_file in reference_files:
            # Convert to Path object
            ref_file = Path(ref_file)
            with fits.open(ref_file) as hdul:
                for hdu in hdul:
                    if isinstance(hdu, fits.BinTableHDU):
                        table = Table.read(hdu)
                        expected_counts[hdu.name] = expected_counts.get(hdu.name, 0) + len(table)
    
    # Now process the test file
    print(f"\nVerifying: {test_file.name}")
    with fits.open(test_file) as hdul:
        # Print basic file info
        print(f"  - File size: {test_file.stat().st_size / 1e6:.2f} MB")
        
        total_actual = 0
        total_expected = sum(expected_counts.values()) if expected_counts else None
        
        for hdu in hdul:
            print(f"\n  HDU {hdu.name}:")
            
            # Handle Primary HDU (metadata)
            if isinstance(hdu, fits.PrimaryHDU):
                print("  - Primary HDU (no table)")
                
            # Handle Binary Table HDUs (data tables)
            elif isinstance(hdu, fits.BinTableHDU):
                table = Table.read(hdu)
                actual_len = len(table)
                actual_counts[hdu.name] = actual_len
                total_actual += actual_len
                
                print(f"  - Rows: {actual_len}")
                
                # Compare with expected if available
                if hdu.name in expected_counts:
                    expected = expected_counts[hdu.name]
                    if actual_len == expected:
                        print(f"    ✓ Matches reference ({expected} rows)")
                    else:
                        print(f"    ✗ Discrepancy (reference has {expected} rows)")
                elif expected_counts:
                    print("    ! No reference data for this table")
        
        # Print summary
        print("\n  SUMMARY:")
        print(f"  - Total rows in test file: {total_actual}")
        
        if total_expected is not None:
            print(f"  - Total expected from references: {total_expected}")
            if total_actual == total_expected:
                print("  ✓ Total row count matches references")
            else:
                print(f"  ✗ Total row count discrepancy (difference: {total_actual - total_expected})")


In [3]:
# Set up paths for where the fits data files are located
data_dir = Path("../data")
fits_files = sorted(data_dir.glob("*.fits"))

# Point this to where the additional file to be concatenated is located
additional_file = Path("/home/damian/padre_meddea/scripts/padre_meddea_l1test_eventlist_20250504T105954_v0.1.0.fits")

In [4]:

print("\n" + "="*50)
print("CONCAT FILES VERIFICATION")
print("="*50)

# Concatenate files into new daily file and verify
output_path = concatenate_daily_fits(fits_files)
verify_fits_file(output_path, fits_files)


CONCAT FILES VERIFICATION


2025-05-12 16:26:47 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-BEG' types <class 'str'> and <class 'str'>, choosing DATE-BEG='2025-05-04T08:03:30.385'
2025-05-12 16:26:47 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-END' types <class 'str'> and <class 'str'>, choosing DATE-END='2025-05-04T08:18:30.836'
2025-05-12 16:26:47 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'ORIGFILE' types <class 'str'> and <class 'str'>, choosing ORIGFILE='PADREMDA0_250504080335.DAT'
2025-05-12 16:26:47 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-AVG' types <class 'str'> and <class 'str'>, choosing DATE-AVG='2025-05-04T08:11:05.938'
2025-05-12 16:26:47 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'CHECKSUM' types <class 'str'> and <class 'str'>, choosing CHECKSUM='6hmb8hlb6hlb6hlb'
2025-05-12 16:26:47 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATASUM' types <class '


Analyzing 9 reference files...

Verifying: padre_meddea_l1_eventlist_20250504T000000_v0.1.0.fits
  - File size: 304.36 MB

  HDU PRIMARY:
  - Primary HDU (no table)

  HDU SCI:
  - Rows: 11574576
    ✓ Matches reference (11574576 rows)

  HDU PKT:
  - Rows: 81035
    ✓ Matches reference (81035 rows)

  SUMMARY:
  - Total rows in test file: 11655611
  - Total expected from references: 11655611
  ✓ Total row count matches references


In [5]:
print("\n" + "="*50)
print("ADDITIONAL REFERENCE FILE VERIFICATION")
print("="*50)

fits_files.append(additional_file)  # Add additional file to the list
output_path = concatenate_daily_fits([additional_file], output_path)  # Concatenate again
verify_fits_file(output_path, fits_files)  # Compare with additional reference file


ADDITIONAL REFERENCE FILE VERIFICATION


2025-05-12 16:26:54 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-BEG' types <class 'str'> and <class 'str'>, choosing DATE-BEG='2025-05-04T10:59:54.456'
2025-05-12 16:26:54 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-END' types <class 'str'> and <class 'str'>, choosing DATE-END='2025-05-04T11:15:32.064'
2025-05-12 16:26:54 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'ORIGFILE' types <class 'str'> and <class 'str'>, choosing ORIGFILE='PADREMDA0_250504105959.DAT'
2025-05-12 16:26:54 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATE-AVG' types <class 'str'> and <class 'str'>, choosing DATE-AVG='2025-05-04T11:07:42.414'
2025-05-12 16:26:54 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'CHECKSUM' types <class 'str'> and <class 'str'>, choosing CHECKSUM='X3FiY0CgX0CgX0Cg'
2025-05-12 16:26:54 - astropy - WARNING: MergeConflictWarning: Cannot merge meta key 'DATASUM' types <class '


Analyzing 10 reference files...

Verifying: padre_meddea_l1_eventlist_20250504T000000_v0.1.0.fits
  - File size: 338.17 MB

  HDU PRIMARY:
  - Primary HDU (no table)

  HDU SCI:
  - Rows: 12860638
    ✓ Matches reference (12860638 rows)

  HDU PKT:
  - Rows: 90010
    ✓ Matches reference (90010 rows)

  SUMMARY:
  - Total rows in test file: 12950648
  - Total expected from references: 12950648
  ✓ Total row count matches references
